In [1]:
import wrangle

import pandas as pd
pd.options.display.max_rows = 100

import matplotlib.pyplot as plt
import seaborn as sns

>## Acquiring data through wrangle.py


In [2]:
df=wrangle.get_zillow_data()

>## Defining the dataset

In [3]:
#seeing what dataset looks like
wrangle.df_summary(df)

---Shape: (52319, 68)

---Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52319 entries, 0 to 52318
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            52319 non-null  int64  
 1   parcelid                      52319 non-null  int64  
 2   airconditioningtypeid         13615 non-null  float64
 3   architecturalstyletypeid      70 non-null     float64
 4   basementsqft                  47 non-null     float64
 5   bathroomcnt                   52319 non-null  float64
 6   bedroomcnt                    52319 non-null  float64
 7   buildingclasstypeid           0 non-null      float64
 8   buildingqualitytypeid         33654 non-null  float64
 9   calculatedbathnbr             52184 non-null  float64
 10  decktypeid                    389 non-null    float64
 11  finishedfloor1squarefeet      4371 non-null   float64
 12  calculatedfinishedsquarefeet 

## Handling nulls and missing values:

In [4]:
#setting up missing values function, to catch only the %%% of data that
#is already there, and not missing.

#checking nulls by columns of the percent of nulls found:

wrangle.nulls_by_columns(df).sort_values(by='percent', ascending=False)


,count,percent
buildingclassdesc,52319,1.000000
buildingclasstypeid,52319,1.000000
finishedsquarefeet15,52319,1.000000
finishedsquarefeet13,52319,1.000000
storytypeid,52272,0.999102
basementsqft,52272,0.999102
storydesc,52272,0.999102
yardbuildingsqft26,52256,0.998796
architecturalstyletypeid,52249,0.998662
architecturalstyledesc,52249,0.998662


In [5]:
#now let's look at nulls by rows:
wrangle.nulls_by_rows(df)

n_missing  percent_missing
23         0.338235               2
24         0.352941              12
25         0.367647              11
26         0.382353              30
27         0.397059             177
28         0.411765             390
29         0.426471            2527
30         0.441176            2199
31         0.455882            5989
32         0.470588            8885
33         0.485294           11967
34         0.500000           11157
35         0.514706            3469
36         0.529412            4138
37         0.544118            1020
38         0.558824             223
39         0.573529              29
40         0.588235              15
41         0.602941               6
42         0.617647               7
43         0.632353              10
44         0.647059              46
45         0.661765               6
46         0.676471               1
47         0.691176               1
48         0.705882               2
dtype: int64

In [6]:
#using 50/50 first to see what dataset looks like:
print('Before dropping nulls, %d rows, %d cols' % df.shape)

#dropping the cols & rows that do not meet the 50/75 standard
df = wrangle.handle_missing_values(df, prop_required_column=.5, prop_required_row=.75)
print('After dropping nulls, %d rows, %d cols' % df.shape)

Before dropping nulls, 52319 rows, 68 cols
After dropping nulls, 52241 rows, 34 cols


In [7]:
df.isnull().sum()

id                                  0
parcelid                            0
bathroomcnt                         0
bedroomcnt                          0
buildingqualitytypeid           18588
calculatedbathnbr                  61
calculatedfinishedsquarefeet        8
finishedsquarefeet12              172
fips                                0
fullbathcnt                        61
heatingorsystemtypeid           18392
latitude                            0
longitude                           0
lotsizesquarefeet                 357
propertycountylandusecode           0
propertylandusetypeid               0
propertyzoningdesc              18497
rawcensustractandblock              0
regionidcity                     1025
regionidcounty                      0
regionidzip                        19
roomcnt                             0
unitcnt                         18480
yearbuilt                          41
structuretaxvaluedollarcnt         72
taxvaluedollarcnt                   1
assessmentye

#### For my next null steps:
    - I'll drop all the larger null columns.
    - I'll drop all columns that reiterate information too.
    - I'll average the smaller null columns with fillna

In [8]:
#creating a function to remove unwanted columns (*may add/change later)

cols_to_remove = ['propertyzoningdesc','buildingqualitytypeid','heatingorsystemtypeid','unitcnt','heatingorsystemdesc','calculatedbathnbr','id','finishedsquarefeet12','fullbathcnt','structuretaxvaluedollarcnt','landtaxvaluedollarcnt','taxamount','regionidcity']
def remove_columns(df, cols_to_remove): 
    df = df.drop(columns=cols_to_remove)
    return df

df= remove_columns(df, cols_to_remove )
df.head()

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,propertylandusetypeid,...,regionidcounty,regionidzip,roomcnt,yearbuilt,taxvaluedollarcnt,assessmentyear,censustractandblock,logerror,transactiondate,propertylandusedesc
0,14297519,3.5,4.0,3100.0,6059.0,33634931.0,-117869207.0,4506.0,122,261.0,...,1286.0,96978.0,0.0,1998.0,1023282.0,2016.0,6.059063e+13,0.025595,2017-01-01,Single Family Residential
1,17052889,1.0,2.0,1465.0,6111.0,34449266.0,-119281531.0,12647.0,1110,261.0,...,2061.0,97099.0,5.0,1967.0,464000.0,2016.0,6.111001e+13,0.055619,2017-01-01,Single Family Residential
2,14186244,2.0,3.0,1243.0,6059.0,33886168.0,-117823170.0,8432.0,122,261.0,...,1286.0,97078.0,6.0,1962.0,564778.0,2016.0,6.059022e+13,0.005383,2017-01-01,Single Family Residential
3,12177905,3.0,4.0,2376.0,6037.0,34245180.0,-118240722.0,13038.0,0101,261.0,...,3101.0,96330.0,0.0,1970.0,145143.0,2016.0,6.037300e+13,-0.103410,2017-01-01,Single Family Residential
4,12095076,3.0,4.0,2962.0,6037.0,34145202.0,-118179824.0,63000.0,0101,261.0,...,3101.0,96293.0,0.0,1950.0,773303.0,2016.0,6.037461e+13,-0.001011,2017-01-01,Single Family Residential


In [9]:
df.isnull().sum()

parcelid                          0
bathroomcnt                       0
bedroomcnt                        0
calculatedfinishedsquarefeet      8
fips                              0
latitude                          0
longitude                         0
lotsizesquarefeet               357
propertycountylandusecode         0
propertylandusetypeid             0
rawcensustractandblock            0
regionidcounty                    0
regionidzip                      19
roomcnt                           0
yearbuilt                        41
taxvaluedollarcnt                 1
assessmentyear                    0
censustractandblock             108
logerror                          0
transactiondate                   0
propertylandusedesc               0
dtype: int64

In [20]:
df.lotsizesquarefeet.mean()

11249.861016883819

In [22]:
#looking at value_counts() for each leftover nulls
df.lotsizesquarefeet.value_counts()

6000.0      873
5000.0      413
7200.0      306
7000.0      285
6500.0      271
           ... 
20038.0       1
24886.0       1
2527.0        1
126718.0      1
47405.0       1
Name: lotsizesquarefeet, Length: 14253, dtype: int64

In [11]:
#I'll need to look at how to fillin or sort missing values and determine which columns I want to keep

#dropping columns:
#finishedsquarefeet12              246
#finishedsquarefeet13            52319
#finishedsquarefeet15            52319
#finishedsquarefeet50            47948
#finishedsquarefeet6             52154
#calculatedbathnbr                 135
#id                                  0
#architecturalstyletypeid        52249


